In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install evaluate
!pip install seqeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.5.1
    Uninstalling fsspec-2025.5.1:
      Successfully uninstalled fsspec-2025.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidi

In [3]:
%%writefile hw_02_ner_ddp.py

import os
import numpy as np
from transformers import AutoModelForTokenClassification, AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval   # pip install seqeval
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp

# 设置分布式环境
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)

# 清理分布式环境
def cleanup():
    dist.destroy_process_group()

def train(rank, world_size):
    setup(rank, world_size)
    # 数据集
    ds = load_dataset('nlhappy/CLUE-NER')
    # entity_index
    entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', \
               'company', 'scene', 'book', 'organization', 'government'})
    tags = ['O']
    for entity in entites[1:]:
        tags.append('B-' + entity.upper())
        tags.append('I-' + entity.upper())
    
    entity_index = {entity:i for i, entity in enumerate(entites)}

    tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
    
    def entity_tags_proc(item):
        # item即是dataset中记录
        text_len = len(item['text'])  # 根据文本长度生成tags列表
        tags = [0] * text_len    # 初始值为‘O’
        # 遍历实体列表，所有实体类别标记填入tags
        entites = item['ents']
        for ent in entites:
            indices = ent['indices']  # 实体索引
            label = ent['label']   # 实体名
            tags[indices[0]] = entity_index[label] * 2 - 1
            for idx in indices[1:]:
                tags[idx] = entity_index[label] * 2
        return {'ent_tag': tags}
    
    # 使用自定义回调函数处理数据集记录
    ds1 = ds.map(entity_tags_proc)
    
    def data_input_proc(item):
        # 输入文本先拆分为字符，再转换为模型输入的token索引
        batch_texts = [list(text) for text in item['text']]
        # 导入拆分为字符的文本列表时，需要设置参数is_split_into_words=True
        input_data = tokenizer(batch_texts, truncation=True, add_special_tokens=False, max_length=512, 
                               is_split_into_words=True, padding='max_length')
        input_data['labels'] = [tag + [0] * (512 - len(tag)) for tag in item['ent_tag']]
        return input_data
        
    
    ds2 = ds1.map(data_input_proc, batched=True)  # batch_size 1000
    
    local_rank = rank
    
    id2lbl = {i:tag for i, tag in enumerate(tags)}
    lbl2id = {tag:i for i, tag in enumerate(tags)}
    
    model = AutoModelForTokenClassification.from_pretrained('google-bert/bert-base-chinese', 
                                                            num_labels=21,
                                                            id2label=id2lbl,
                                                            label2id=lbl2id)
    model.to(local_rank)
    
    # args = TrainingArguments(
    #     output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
    #     num_train_epochs = 3,    # 训练 epoch
    #     save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
    #     per_device_train_batch_size=16,  # 训练批次
    #     per_device_eval_batch_size=16,
    #     eval_strategy="epoch",      # 每个epoch结束后进行评估
    #     #save_strategy="epoch",      # 每个epoch结束后保存模型
    #     #load_best_model_at_end=True,    # 训练结束后加载最佳模型
    #     #metric_for_best_model="f1",     # 使用f1分数作为评判最佳模型的标准
    #     #report_to='tensorboard',  # 训练输出记录
    #     local_rank=local_rank,   # 当前进程 RANK
    #     fp16=True,               # 使用混合精度
    #     lr_scheduler_type='linear',  # 动态学习率
    #     warmup_steps=100,        # 预热步数
    #     ddp_find_unused_parameters=False  # 优化DDP性能
    # )
    args = TrainingArguments(
        output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
        num_train_epochs = 3,    # 训练 epoch
        save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
        per_device_train_batch_size=8,  # 训练批次
        per_device_eval_batch_size=8,
        report_to='tensorboard',  # 训练输出记录
        eval_strategy="epoch",
        local_rank=local_rank,   # 当前进程 RANK
        fp16=True,               # 使用混合精度
        lr_scheduler_type='linear',  # 动态学习率
        warmup_steps=100,        # 预热步数
        ddp_find_unused_parameters=False  # 优化DDP性能
    )
    
    def compute_metric(result):
        # result 是一个tuple (predicts, labels)
        
        # 获取评估对象
        seqeval = evaluate.load('seqeval')
        predicts,labels = result
        predicts = np.argmax(predicts, axis=2)
        
        # 准备评估数据
        predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                     for ps,ls in zip(predicts,labels)]
        results = seqeval.compute(predictions=predicts, references=labels)
    
        return results
    
    data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, padding=True)
    
    trainer = Trainer(
        model,
        args,
        train_dataset=ds2['train'],
        eval_dataset=ds2['validation'],
        data_collator=data_collator,
        compute_metrics=compute_metric
    )
    
    trainer.train()

    # 在trainer.train()之后添加
    if rank == 0:  # 只有主进程保存模型
        trainer.save_model("/kaggle/output/ner_final_model")  # 保存完整模型
        tokenizer.save_pretrained("/kaggle/output/ner_final_model")  # 保存tokenizer

def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)

if __name__ == "__main__":
    main()

Writing hw_02_ner_ddp.py


In [4]:
!python hw_02_ner_ddp.py

2025-07-30 07:10:13.163042: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753859413.386428      63 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753859413.449972      63 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-30 07:10:38.406891: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-30 07:10:38.406930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:17538

In [5]:
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer

def predict_entities(text, model, tokenizer, id2label, device):
    # 准备输入并移动到指定设备
    inputs = tokenizer(
        list(text), 
        is_split_into_words=True, 
        return_tensors="pt", 
        padding=True, 
        truncation=True
    ).to(device)  # 将输入数据移至GPU/CPU
    
    # 推理
    with torch.no_grad():  # 禁用梯度计算，节省内存并加速
        outputs = model(** inputs)
    
    # 获取预测结果
    predictions = torch.argmax(outputs.logits, dim=2)
    predicted_labels = [id2label[p.item()] for p in predictions[0]]
    
    # 组合结果
    result = list(zip(list(text), predicted_labels))
    return result

# 检查是否有可用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 加载模型和tokenizer
model_path = "/kaggle/output/ner_final_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

# 将模型移动到GPU
model = model.to(device)
# 设置为评估模式（关闭dropout等训练特定层）
model.eval()

# 准备标签映射
entites = ['O'] + list({'movie', 'name', 'game', 'address', 'position', 
                        'company', 'scene', 'book', 'organization', 'government'})
tags = ['O']
for entity in entites[1:]:
    tags.append('B-' + entity.upper())
    tags.append('I-' + entity.upper())
id2label = {i: tag for i, tag in enumerate(tags)}

# 测试文本
test_text = "张艺谋导演的电影《活着》改编自余华的小说"

# 预测
results = predict_entities(test_text, model, tokenizer, id2label, device)

# 打印结果
for char, label in results:
    print(f"{char}: {label}")

使用设备: cuda


2025-07-30 08:19:37.114368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753863577.139198      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753863577.146581      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


张: O
艺: B-ADDRESS
谋: I-ADDRESS
导: I-ADDRESS
演: B-NAME
的: I-NAME
电: O
影: O
《: O
活: B-GAME
着: I-GAME
》: I-GAME
改: I-GAME
编: O
自: O
余: O
华: B-ADDRESS
的: I-ADDRESS
小: O
说: O
